In [47]:
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("bergum/xtremedistil-l6-h384-go-emotion")
model_emotion = AutoModelForSequenceClassification.from_pretrained("bergum/xtremedistil-l6-h384-go-emotion")


classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', \
                      return_all_scores=True)

classifier_emotion = pipeline("text-classification", model=model_emotion, tokenizer=tokenizer,return_all_scores=True)



from nltk.corpus import stopwords
stopwords_dict = {word: 1 for word in stopwords.words("english")}
import emoji
import regex as re

import tweepy
import time
import random
import pandas as pd
print(tweepy.__version__)

4.12.1


In [ ]:
#SETUP TWEEPY

CONSUMER_KEY ='Yourkey'
CONSUMER_SECRET='Yourkey'
ACCESS_TOKEN='Yourkey'
ACCESS_TOKEN_SECRET='Yourkey'

BT='YourkeyYourkey'

client2 = tweepy.Client(bearer_token=BT)

client = tweepy.Client(consumer_key=CONSUMER_KEY,
                       consumer_secret=CONSUMER_SECRET,
                       access_token=ACCESS_TOKEN,
                       access_token_secret=ACCESS_TOKEN_SECRET,
                      bearer_token=BT)


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)



#SETUP AUTHORIZATION OF ANOTHER TWITTER AC

consumer_key = CONSUMER_KEY
consumer_secret = CONSUMER_SECRET
callback_uri = 'oob'
api = tweepy.API(auth)

auth = tweepy.OAuth1UserHandler(
   consumer_key, consumer_secret,
   ACCESS_TOKEN, ACCESS_TOKEN_SECRET
)

oauth1_user_handler = tweepy.OAuth1UserHandler(
    consumer_key, consumer_secret,
    callback=callback_uri
)

print(oauth1_user_handler.get_authorization_url(signin_with_twitter=True))

token =input("please enter tokenfrom this url")
#token=6580548

access_token, access_token_secret = oauth1_user_handler.get_access_token(token)

tweet_client = tweepy.Client(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token, 
    access_token_secret=access_token_secret
)



In [288]:

#SETUP SEARCH WORDS
searchlist=["i miss you today", "worst feeling ever", "this is not ok", "not cool",\
"heavy heart","im so mad","like i failed" , "i give up" , " makes me crazy"\
"start over again","sad today", "just lonely", \
"feel like shit","so bummed" ," i dont care" ," go to hell"]


messages= ["🤗❤️ ((hugs)) ☕️🍪 ","HUGS!! from me 🤗😊🍪❤️" ,"((hugs)) 🥰😍☕️🍪","♥️🙏🏽", "Sending you love 🤗♥️☕️🍪", "Many ((hugs)) from me  🤗♥️☕️🍪" , "♥️","🤗♥️","♥️ u" , "lots of love ♥️ ",           "🤗❤️ No Sad, Happy Happy 🤗♥️", "♥️", "No Sad 🍪❤️", "😊 no sad ☕️"
          ]

In [289]:
tweetid_l, tweet_text_l, userid_l, screenname_l, username_l, location_l, description_l, followers_count_l,friends_count_l, created_at_l, website_l= [],[],[],[],[],[],[],[],[],[],[]

for s in range(len(searchlist)):
    response= client.search_recent_tweets(query = searchlist[s] ,max_results=30,expansions = ["author_id"],user_fields=['profile_image_url'])
    results= response[-1]['result_count']
    print("found", results)
    for i in range(results-20):
        tweetid_l.append(response[0][i]["id"])

        tweet_text_l.append(response[0][i]["text"])

        #print(i)
        userid_l.append(response[1]["users"][i]["id"])

        username_l.append(response[1]["users"][i]["name"])

        user_info =api.lookup_users( screen_name=None, user_id=[response[-3]["users"][i]["id"]])

        #print(user_info)                                                     
        screenname_l.append(user_info[0]._json["screen_name"])                                                    
        description_l.append(user_info[0]._json["description"] ) 
        friends_count_l.append(user_info[0]._json["friends_count"] ) 
        followers_count_l.append(user_info[0]._json["followers_count"])
        location_l.append(user_info[0]._json["location"])



leads_bd= pd.concat([ pd.Series(tweetid_l),pd.Series(tweet_text_l),pd.Series(userid_l),pd.Series(username_l),pd.Series(screenname_l), pd.Series(description_l),            pd.Series(friends_count_l),            pd.Series(followers_count_l),            pd.Series(location_l) ],axis=1)

leads_bd.columns=["tweetid","tweet_text",'user_id','username','screenname','description', 'friends_count','follower_count','location']



print("found tweets", leads_bd.shape )
print("GETTING REPLIES METRICS _ PUBLIC METRICS")

retweet,reply,like,quote =[],[],[],[]

for i in range(leads_bd.shape[0]):
    time.sleep(random.sample([2,1],k=1)[0])
    response  = client.get_tweets(ids=leads_bd["tweetid"][i],tweet_fields=["public_metrics"], expansions=["attachments.media_keys"],media_fields=["public_metrics"])
    try:
        retweet.append(response.data[0].public_metrics['retweet_count'])
        reply.append(response.data[0].public_metrics['reply_count'])
        like.append(response.data[0].public_metrics['like_count'])
        quote.append(response.data[0].public_metrics['quote_count'])
    except Exception:
        retweet.append(0)
        reply.append(0)
        like.append(0)
        quote.append(0)


metrics_db=pd.concat([pd.Series(retweet),pd.Series(reply),pd.Series(like),pd.Series(quote)],axis=1)
metrics_db.columns=["retweet","reply","like","quote"]
print("FOUND METRICS", metrics_db.shape)
all_searches= pd.concat([metrics_db,leads_bd],axis=1)

#MIX UP THE DB
all_searches_v1_f=all_searches.sample(frac=1)

#reducing the high volume tweets
all_searches_v1_f2 = all_searches_v1_f.loc[all_searches_v1_f['retweet']<=10]
all_searches_v2 = all_searches_v1_f2.loc[all_searches_v1_f2['like']<=10]
all_searches_v3 = all_searches_v2.loc[all_searches_v2['reply']<=10]

print("FILTERED TO NOT VERY POPULAR",all_searches.shape,all_searches_v3.shape)


#RESET INDEX
all_searches_v3.reset_index(inplace=True)
print("THIS IS THE TWEET DB I WILL USE")
print(all_searches_v3.head(3))



#MAKE TWEET IDS
topic_text= all_searches_v3["tweet_text"].values



found 30
found 30
found 30
found 29
found 29
found 29
found 30
found 30
found 0
found 29
found 28
found 29
found 29
found 30
found 30
found tweets (132, 9)
GETTING REPLIES METRICS _ PUBLIC METRICS
FOUND METRICS (132, 4)
FILTERED TO NOT VERY POPULAR (132, 13) (89, 13)
THIS IS THE TWEET DB I WILL USE
   index  retweet  reply  like  quote              tweetid  \
0     20        0      0     0      0  1594371389582372864   
1     98        0      0     0      0  1594371237907668992   
2    113        0      1     0      0  1594371422633205762   

                                          tweet_text              user_id  \
0  @CharlotteKhuma3 @HiIamNicholas @piersmorgan N...            271100615   
1  Chris caption just made me even more upset. Li...  1299856704696389633   
2  also want to clarify i dont care if you ship t...  1517604124388474888   

  username       screenname                      description  friends_count  \
0        F  Sincerely_farah  I will criticise you. 🇰🇼305  ♍️   

In [290]:

#Clean Tweets
topic_clean=[]

for i in range(len(topic_text)):
    #print(topic_text[i].text,i)

    s = topic_text[i]
    s = " ".join([word for word in s.split() if word not in stopwords_dict])
    s =re.sub('RT', '', s)
    s =re.sub('https://\w+.\w+/\w+', '', s)
    s =re.sub('\@\w+','', s)

    s =re.sub('\#\w+', '', s)
    s=re.sub('\n', '', s)
    s =emoji.replace_emoji(s)
    s =re.sub('[^A-Za-z0-9]+', ' ', s)
    s =re.sub('\n+ ', '', s)
    topic_clean.append(s)



messages1= ["🤗❤️ ((hugs)) ☕️🍪 ","HUGS!! from me 🤗😊🍪❤️" ,"((hugs)) 🥰😍☕️🍪","♥️🙏🏽", "Sending you love 🤗♥️☕️🍪",\
          "Many ((hugs)) from me  🤗♥️☕️🍪" , "♥️","🤗♥️","♥️ u" , "lots of love ♥️ ", \
          "🤗❤️ No Sad, Happy Happy 🤗♥️", "♥️", "No Sad 🍪❤️", "😊 no sad ☕️"
          ]

messages2= ["Just Relax 🍬☕️🍪 ","Sending you smiles 🍬☕️🍪 ", "It's going to be ok 🧸" ,"😊Relax 😊🍬"]

messages3= ["Sending sunshine 🤗♥️", "🍬I understand 🧸 ", "Your so 🤗😊 Cool🧸", " "]



hug1="hug1.gif"
hug2="hug2.gif"
hug3="hug3.gif"
hug4="hug4.gif"
hug5="hug5.gif"
hug6="hug6.gif"
hug7="hug7.gif"

happy1="happy1.gif"
happy2="happy2.gif"

angry1="angry1.gif"
angry2="angry2.gif"



#classify emotions _12 _BIG LIST
emotion_nothing=['neutral']
emotion_funny=["amusement"]
emotion_thanks=["relief",'gratitude']
emotion_blush=["embarrassment",]
emotion_love=["love",'desire',"caring",'admiration']
emotion_fear=["fear","nervousness",]
emotion_happy=["optimism",'approval','pride',"excitement"]
emotion_confused=["suprise","confusion"]
emotion_angry=["anger ","disgust","annoyance","disapproval"]
emotion_sad=["sadness",'disappointment']
emotin_greif =["greif","remorse"]
emotin_notlisted =["notlisted","na"]
emotion_list_12 =[emotin_greif,emotion_sad,emotion_angry,emotion_confused,emotion_happy,emotion_fear,emotion_love,\
              emotion_blush,emotion_thanks,emotion_funny,emotion_nothing,emotin_notlisted]
emotion_list_name =["emotin_greif","emotion_sad","emotion_angry","emotion_confused",\
                    "emotion_happy","emotion_fear","emotion_love","emotion_blush","emotion_thanks","emotion_funny","emotion_nothing","emotin_notlisted"]




message_greif =["I'm here with you today ♥️",\
                "sorry ☹️ ♥️🧸",\
                "Stay strong ♥️ and wishing you better days"]

message_sad= ["going for a nice walk sometimes helps cheer me up 😊♥️",\
              "Stay strong ♥️",\
              "🌞and sunshine is just right the corner , summer 2023 " ,\
              "good things take time" ,]

message_angry= ["🍪 relax  and just calm down","just breathe😅",\
                " just relax 🍬 "  ,"good things take time"  ]

message_confused= ["flip a coin 😂","wondering as well","a mystery 😅" ,"idk 😊♥️" ,"confusing 😊"]

message_happy= ["what a beautiful day🌞 ", " positive🌞" , "keep smiling ☺️😊" ,]

message_fear= ["im not scared", " So not afraid", "your stronger than this, no fear"]

message_love= ["😊♥️","♥️","love is precious","so precious"]

message_blush= ["you look good today ","hello cutie", "your very good loking", "i like you too", "hello ☺️😊"]

message_thanks= ["what a beautiful day🌞 ", " positive🌞" , "keep smiling ☺️😊" ,]

message_funny= ["😂","strangely funny😂","lol 😂" ,"a little amusing😂😂","😂😅"]

message_nothing= ["🍪☕️🍬","🍪","☕️","♥️","c((hugs))","♥️☕️ for you"]

message_notlisted= ["♥️","♥️☕️"]


message_list=[message_greif,message_sad,message_angry,message_confused,\
                    message_happy,message_fear,message_love,\
              message_blush,message_thanks,message_funny,message_nothing,emotin_notlisted]




In [291]:
len(topic_clean)

89

In [280]:
emotion_list_name

['emotin_greif',
 'emotion_sad',
 'emotion_angry',
 'emotion_confused',
 'emotion_happy',
 'emotion_fear',
 'emotion_love',
 'emotion_blush',
 'emotion_thanks',
 'emotion_funny',
 'emotion_nothing',
 'emotin_notlisted']

In [297]:
emotion_list=[]
media_list=[]
message_to_post=[]

for e in topic_clean:
    prediction = classifier_emotion(e)
    emotion= pd.DataFrame(prediction[0]).sort_values(by="score",ascending=False)[:1]["label"].values[0].split(" ")[0]
    emotion_list.append(emotion)


    d=""
    i=0
    for d in emotion_list_12:
        #print(emotion_list[i])
        if emotion in d:
            detected_list = d
            message_to_post.append((random.sample(message_list[i],k=1))[0])
            #print(detected_list)
            break
        i+=1
    
    print(i)
    
    media_list.append(random.sample([hug1,hug2,hug3],k=1))
#print(message_to_post)
#print(emotion_list)

10
2
10
2
9
1
10
8
6
10
9
10
10
10
10
10
2
10
10
10
4
10
10
10
10
2
10
10
2
10
2
10
10
2
10
10
6
10
10
9
10
10
10
6
1
1
1
1
10
10
6
2
2
12
1
6
10
1
10
10
10
10
1
10
10
10
2
10
9
10
1
8
6
6
2
10
10
2
1
10
10
10
2
2
9
6
2
10
2


In [298]:
pd.DataFrame(prediction[0]).sort_values(by="score",ascending=False)[:1]["label"].values[0].split(" ")[0]

'annoyance'

In [256]:
len(emotion_list)

31

In [176]:
len(message_list)

12

In [170]:
len(emotion_list_12)

12

In [183]:
len(emotion_list_name)

12

In [182]:
emotion_list_name

['emotin_greif',
 'emotion_sad',
 'emotion_angry',
 'emotion_confused',
 'emotion_happy',
 'emotion_fear',
 'emotion_love',
 'emotion_blush',
 'emotion_thanks',
 'emotion_funny',
 'emotion_nothing',
 'emotin_notlisted']

In [210]:

detected_list=[]
i=0
for d in emotion_list_12:
    #print(emotion_list[i])
    if emotion in d:
        detected_list.append(emotion_list_name[i])
        break
    m_index=i
    i+=1
if len(detected_list) == 0:
    detected_list.append("notlisted")

In [232]:
random.sample(message_list[11],k=1)[0]

'notlisted'

In [149]:
message_list[9]

['😂', 'strangely funny😂', 'lol 😂', 'a little amusing😂😂', '😂😅']

In [299]:
pd.concat([pd.Series(emotion_list),pd.Series(topic_clean),pd.Series(message_to_post),],axis=1)

,0,1,2
0,neutral,Not necessarily grandchildren may also look d...,🍪
1,annoyance,Chris caption made even upset Like man could v...,just relax 🍬
2,neutral,also want clarify dont care ship wont hate twe...,🍪
3,annoyance,gala kharab hona worst feeling ever,just relax 🍬
4,amusement,takemob tried colour like pro failed im laugh,a little amusing😂😂
...,...,...,...
84,amusement,finishing midnight fight express need dlc need...,♥️
85,admiration,You cool better perk bitch,good things take time
86,annoyance,don t feel like i ll never get certain shit,☕️
87,neutral,November 26 dont care really,just relax 🍬


In [233]:
message_to_post

['good things take time',
 'a little amusing😂😂',
 'what a beautiful day🌞 ',
 'lol 😂',
 'sorry ☹️ ♥️🧸',
 'lol 😂',
 'keep smiling ☺️😊',
 'strangely funny😂',
 'good things take time',
 'Stay strong ♥️ and wishing you better days',
 'a little amusing😂😂',
 'strangely funny😂',
 'sorry ☹️ ♥️🧸',
 "I'm here with you today ♥️",
 'lol 😂',
 "I'm here with you today ♥️",
 'your stronger than this, no fear',
 'a little amusing😂😂',
 'going for a nice walk sometimes helps cheer me up 😊♥️',
 'Stay strong ♥️ and wishing you better days',
 'sorry ☹️ ♥️🧸',
 'lol 😂',
 'sorry ☹️ ♥️🧸',
 'strangely funny😂',
 'im not scared',
 'sorry ☹️ ♥️🧸',
 '😂😅',
 'good things take time',
 'Stay strong ♥️ and wishing you better days',
 '😂',
 'going for a nice walk sometimes helps cheer me up 😊♥️']

In [ ]:

#MAKE TWEET IDS
tweet_ids= all_searches_v3["tweetid"].values

#MediAPI
media_api = tweepy.API(oauth1_user_handler )


#NOW WE WILL SEND HUGS REPLIES


for i in range(len(tweet_ids)):
    try:
        e_reply= message_to_post[i]
        #reply = random.sample(messages,k=1)[0] 
        #print(all_searches_v3["tweet_text"].loc[i])  

        print(tweet_ids[i])
        print(e_reply)
        #print("HUGGED SOMEONE", i,len(tweet_ids))

       
        print("I REPLIED", i,len(tweet_ids))
        time.sleep(random.sample([11,18,9,13],k=1)[0])


        if media_list[i]=="skip":
          

        if emotion_list[i]=="anger"|"annoyance"|sadness:

            print("GIPHY",emotion_list[i])

            media = media_api.media_upload(angry1).media_id
            print(media)



        if random.sample([297,13,0,297,15,49],k=1)[0] ==297:
            tweet_client.like(tweet_id=tweet_ids[i])
        print(all_searches_v3["tweet_text"][i])

        time.sleep(random.sample([11,18,9,49,13],k=1)[0])

    except Exception:
        print("ERROR")
        pass

    

print("SET ONE DONE")
print (z)
time.sleep(297)
time.sleep(297)
time.sleep(297)


'''
#classify emotions
emotion_list=[]
media_list=[]
message_to_post=[]

for e in topic_clean:
    prediction = classifier(e)
    emotion= pd.DataFrame(prediction[0]).sort_values(by="score",ascending=False).reset_index().loc[0]["label"]
    emotion_list.append(emotion)

    if emotion == "joy":
        m_to_post=random.sample([happy1,happy2],k=1)[0]
        message=random.sample(messages3,k=1)[0]
    elif emotion == "anger":
        m_to_post=random.sample([angry1,angry2],k=1)[0]
        message=random.sample(messages2,k=1)[0]
    elif emotion == "love":
        m_to_post=random.sample([hug1,hug2,hug3,hug4,hug5],k=1)[0]
        message=random.sample(messages1,k=1)[0]

    else:
        message=random.sample(messages1,k=1)[0]
        m_to_post="skip"

    message_to_post.append(message)    
    media_list.append(m_to_post)


In [ ]:
for i in range(25,len(tweet_ids)):
    e_reply= message_to_post[i-7]
    #reply = random.sample(messages,k=1)[0] 
    #print(all_searches_v3["tweet_text"].loc[i])  

    print(tweet_ids[i])
    print(e_reply)
    #print("HUGGED SOMEONE", i,len(tweet_ids))

    tweet_client.create_tweet(in_reply_to_tweet_id=tweet_ids[i],text=e_reply)
    print("I REPLIED", i,len(tweet_ids))
    time.sleep(random.sample([11,18,9,13],k=1)[0])


    if media_list[i]=="skip":
        tweet_client.create_tweet(in_reply_to_tweet_id=tweet_ids[i],text=e_reply)

    if emotion_list[i] in ["anger","annoyance","sadness"]:
        print("GIPHY",emotion_list[i])
        media = media_api.media_upload(angry1).media_id
        print(media)
        tweet_client.create_tweet(in_reply_to_tweet_id=tweet_ids[i],media_ids= [media],text=e_reply)
        

    if random.sample([297,13,0,297,15,49],k=1)[0] ==297:
        tweet_client.like(tweet_id=tweet_ids[i])
    print(all_searches_v3["tweet_text"][i])